In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import copy
import os
import torch
from PIL import Image
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torchvision import utils
%matplotlib inline

In [2]:
path = "/kaggle/input/aml-challenge1"
import pandas as pd

labels_df = pd.read_csv(path+'/train.csv')  # Adjust filename
print(labels_df.head())

                                     id  has_cactus
0  0004be2cfeaba1c0361d39e2b000257b.jpg           1
1  000c8a36845c0208e833c79c1bffedd1.jpg           1
2  000d1e9a533f62e55c289303b072733d.jpg           1
3  0011485b40695e9138e92d0b3fb55128.jpg           1
4  0014d7a11e90b62848904c1418fc8cf2.jpg           1


In [3]:
print(labels_df.shape)
labels_df[labels_df.duplicated(keep=False)]

(17500, 2)


,id,has_cactus


In [4]:
labels_df['has_cactus'].value_counts()

has_cactus
1    13136
0     4364
Name: count, dtype: int64

In [5]:
import torch
torch.manual_seed(0)

from torch.utils.data import Dataset
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms

class pytorch_data(Dataset):
    
    def __init__(self, data_dir, transform, data_type="train"):
        # Get Image File Names
        cdm_data = os.path.join(data_dir, data_type)
        file_names = os.listdir(cdm_data)

        all_image_paths = [os.path.join(cdm_data, f) for f in file_names if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

        print(f"Found {len(all_image_paths)} images in directory.")
        print(f"Sample filenames: {all_image_paths[:1]}")

        # Get Labels
        labels_data = os.path.join("/kaggle/input/aml-challenge1/", "train.csv")
        labels_df = pd.read_csv(labels_data)

        # Normalize index: remove extensions if present
        labels_df['id'] = labels_df['id'].apply(lambda x: os.path.splitext(str(x))[0])
        labels_df.set_index("id", inplace=True)

        print(f"Labels dataframe length: {len(labels_df)}")

        # Extract only valid images (that have a label)
        valid_filenames = []
        labels = []

        for f in all_image_paths:
            filename = os.path.basename(f)  # get only file name
            image_id = os.path.splitext(filename)[0]  # remove extension

            if image_id in labels_df.index:
                valid_filenames.append(f)
                labels.append(labels_df.loc[image_id].values[0])
            else:
                print(f"Warning: image '{filename}' has no matching label in train.csv")

        self.full_filenames = valid_filenames
        self.labels = labels
        self.transform = transform

        print(f"Valid image-label pairs: {len(self.full_filenames)}")
        print(f"First few labels: {self.labels[:5]}")
      
    def __len__(self):
        return len(self.full_filenames)
      
    def __getitem__(self, idx):
        if idx >= len(self.full_filenames):
            raise IndexError(f"Index {idx} out of bounds for dataset of length {len(self.full_filenames)}")

        image = Image.open(self.full_filenames[idx])
        image = self.transform(image)
        return image, self.labels[idx]

In [6]:
# define transformation that converts a PIL image into PyTorch tensors
import torchvision.transforms as transforms
data_transformer = transforms.Compose([transforms.ToTensor(),
                                       transforms.Resize((32,32))])

In [7]:
# Define an object of the custom dataset for the train folder.
data_dir = path+'/train/'
img_dataset = pytorch_data(data_dir, data_transformer, "train")

Found 17500 images in directory.
Sample filenames: ['/kaggle/input/aml-challenge1/train/train/5d3a7d32516a92cc0dc8c52af515eaa4.jpg']
Labels dataframe length: 17500
Valid image-label pairs: 17500
First few labels: [1, 0, 1, 1, 1]


In [8]:
# define transformation that converts a PIL image into PyTorch tensors
data_transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32))
])

In [9]:
# Test a sample
img, label = img_dataset[10]
print(img.shape, torch.min(img), torch.max(img))

torch.Size([3, 32, 32]) tensor(0.2667) tensor(0.8627)


**DATA AUGMENTATION CHE VA A RADDOPPIARE IL NUMERO DI SAMPLE DELLA CLASSE 0 AVENDO ALLA FINE CHE I SAMPLE DELLA CLASSE 0 SONO I 2/3 DEI SAMPLE DELLA CLASSE 1**

In [10]:
# Aumentiamo le immagini della classe 0
from torchvision.transforms import RandomRotation, ToTensor, Resize
from tqdm import tqdm

# Trasformazione per augmentare
augment_transform = transforms.Compose([
    RandomRotation(degrees=10),
    Resize((32, 32)),
    ToTensor()
])

# Trova solo immagini con etichetta 0
images_class0 = [i for i in range(len(img_dataset)) if img_dataset.labels[i] == 0]

# Duplichiamo queste immagini con trasformazione
augmented_images = []
augmented_labels = []

for idx in tqdm(images_class0):
    img_path = img_dataset.full_filenames[idx]
    img = Image.open(img_path)
    augmented_img = augment_transform(img)
    augmented_images.append(augmented_img)
    augmented_labels.append(0)

# Stack immagini originali
original_images = [img_dataset[i][0] for i in range(len(img_dataset))]
original_labels = [img_dataset[i][1] for i in range(len(img_dataset))]

# Combina immagini originali + augmentate
all_images = torch.stack(original_images + augmented_images)
all_labels = torch.tensor(original_labels + augmented_labels)

# Nuovo Dataset custom con dati augmentati
class AugmentedDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Sostituisci img_dataset con quello nuovo
img_dataset = AugmentedDataset(all_images, all_labels)

100%|██████████| 4364/4364 [00:27<00:00, 161.51it/s]


***DIVISIONE NEI TRE SET TRAIN, VALIDATION, TEST DOPO AVER AUMENTATO I DATI DELLA CLASSE 0***

In [11]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Estrai le etichette in modo sicuro
labels = img_dataset.labels  # NON usare img_dataset[i][1]

# Crea una lista di tutti gli indici
all_indices = list(range(len(img_dataset)))

# Split stratificato: Train (70%) e Temp (30%)
train_idx, temp_idx = train_test_split(
    all_indices, test_size=0.3, stratify=labels, random_state=42
)

# Estrai le label corrispondenti agli indici temporanei per secondo split
temp_labels = [labels[i] for i in temp_idx]

# Split stratificato: Validation (15%) e Test (15%) da temp
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, stratify=temp_labels, random_state=42
)

# Crea i subset PyTorch
train_ts = Subset(img_dataset, train_idx)
val_ts = Subset(img_dataset, val_idx)
test_ts = Subset(img_dataset, test_idx)

# Visualizzazione
print("train dataset size:", len(train_ts))
print("validation dataset size:", len(val_ts))
print("test dataset size:", len(test_ts))

train dataset size: 15304
validation dataset size: 3280
test dataset size: 3280


**DATA LOADER FOR SPLITTING IMAGES**

In [12]:
from torch.utils.data import DataLoader

# Training DataLoader
train_dl = DataLoader(train_ts,
                      batch_size=64, 
                      shuffle=True)

# Validation DataLoader
val_dl = DataLoader(val_ts,
                    batch_size=64,
                    shuffle=False)

In [14]:
from torchvision import models
import torch.nn as nn

# Carica ResNet18 senza pesi pre-addestrati
model = models.resnet18(weights=None)

# Modifica l'output per classificazione binaria
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Manda su GPU se disponibile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from torchvision import models
import torch.optim as optim
import torch.nn as nn

# Griglia di iperparametri da esplorare
learning_rates = [1e-3, 1e-4]
optimizers = ['adam', 'sgd']

results = []

for lr in learning_rates:
    for opt_name in optimizers:
        # Inizializza un nuovo modello ResNet18
        model = models.resnet18(weights=None)
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(nn.Linear(num_ftrs, 1), nn.Sigmoid())
        model = model.to(device)

        # Crea ottimizzatore
        if opt_name == 'adam':
            optimizer = optim.Adam(model.fc.parameters(), lr=lr)
        else:
            optimizer = optim.SGD(model.fc.parameters(), lr=lr, momentum=0.9)

        # Loss e scheduler
        criterion = nn.BCELoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2)

        # Allenamento su 3 epoche per tuning veloce
        for epoch in range(15):
            train_loss, train_acc = train_epoch(model, train_dl, optimizer, criterion)
            val_loss, val_acc = evaluate(model, val_dl, criterion)
            scheduler.step(val_loss)

        results.append({
            'lr': lr,
            'optimizer': opt_name,
            'val_loss': val_loss,
            'val_acc': val_acc
        })

        print(f"lr={lr}, opt={opt_name}, val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

lr=0.001, opt=adam, val_loss=0.2906, val_acc=0.8881
